In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
twentyng_data = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'))
data = twentyng_data.data

In [127]:
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [128]:
import re
import math

In [129]:
def get_sentences(data):
    sentences = []
    for each_d in data:
        lines = tokenizer.tokenize(each_d)
        a = []
        for line in lines:
            line = line.strip() # skip the empty line
            line = line.replace('\n',' ')
            a.append(line)
        sentences.append(a)
    return sentences



In [130]:
def get_word_frequency(sentences):
    word_freq = {}
    word_length = 0
    for s in sentences:
        words = s.split()
        for w in words:
            word_length+=1
            x = w.lower()
            word_freq[x] = word_freq.get(x,0) + 1
    
    content_word_tf = dict((w, f / word_length) for w, f in word_freq.items())
    #    return content_word_tf
    
    return content_word_tf
        

In [131]:
def get_all_words(document):
#     words = []
#     for s in document:
#         x = s.split()
#         for e in x:
#             words.append(e.lower())
    words = [w for s in document for w in s.split()]
        
    return words

In [132]:
def get_word_list_sentence(sentences):
    lol = []
    for s in sentences:
        l = []
        word_list = s.split()
        for w in word_list:
            l.append(w.lower())
        lol.append(l)
    return lol

In [133]:
def get_tf_vector(document):
    try:
        tf_vectorizer = CountVectorizer()
        tf = tf_vectorizer.fit_transform(document)
        tf_feature_names = tf_vectorizer.get_feature_names()
        return tf,tf_feature_names
    except: return [None,None]
    
def get_weighted_word_frequency_lda(sentences):
        sentences = ','.join(sentences)
        tf,feature_names = get_tf_vector([sentences])
        lda = LatentDirichletAllocation(n_topics=20).fit(tf)
        doc_topic_vector = lda.transform(tf)
        #print("doc topic vector:",doc_topic_vector.shape)
        topic_word_vector = np.zeros((20,lda.components_.shape[1]),dtype=np.float)
        index_mapping = dict()
        content_word_tf = dict()
        
        for topic_idx, topic in enumerate(lda.components_):
            #print("checking vocab:",len(topic))
            #new_topic = topic.argsort()[:-no_top_words - 1:-1]
            for j in range(len(topic)):
                index_mapping[j] = feature_names[j]
                
                topic_word_vector[topic_idx][j] = topic[j] / np.sum(lda.components_, axis=0)[j]
            
        
        #print("topic_word_vector",topic_word_vector)
        result_mat = np.matmul(doc_topic_vector,topic_word_vector)
        #print("result_mat",result_mat)
        #print("result_mat shape:",result_mat.shape)
        for i in range(result_mat.shape[1]):
            content_word_tf[index_mapping[i]] = result_mat[0][i]
            #print("content_word_tf:",content_word_tf)    
        return content_word_tf

In [134]:
def combine_frequency(list1,list2):
    total_len = len(list1)+len(list2) 
    wf1 = get_word_frequency(list1)
    wf2 = get_word_frequency(list2)
    res_wf = wf1.copy()
    for k in wf2:
            if k in res_wf: 
                res_wf[k] += wf2[k]
            else:
                res_wf[k] = wf2[k]
    for k in res_wf:
        res_wf[k] /= float(total_len)

    return res_wf

In [135]:
def kl_divergence(summary_freq,doc_freq):
    sum_val = 0
    for w in summary_freq:
        frequency = doc_freq.get(w)
        if frequency: 
            sum_val += frequency * math.log(frequency / summary_freq[w])

    return sum_val

In [150]:
def summarize(document,lda):
    #get the word-frequency dictionary for the entire document
    if lda == True:
        try:
            word_freq = get_weighted_word_frequency_lda(document)
        except:
            return None
    else:
        word_freq = get_word_frequency(document)
    summary = []
    
    sentence_list = list(document)
    
    sentence_word_list = get_word_list_sentence(document)
    
    while len(sentence_list) > 0:
        kl_summaries = []
        summary_word_list = get_all_words(summary)
        
        for word_list in sentence_word_list:
            combine_freq = combine_frequency(word_list,summary_word_list)
        
            kl_summaries.append(kl_divergence(combine_freq,word_freq))
        
        delete_index = kl_summaries.index(min(kl_summaries))
        best_sentence = sentence_list.pop(delete_index)
        
        del sentence_word_list[delete_index]

        summary.append(best_sentence)
    return summary[:5]

    
    


In [137]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [138]:
sentences = get_sentences(data)

In [151]:
KL_summaries_20ng = []
for i in range(len(sentences)):
    each_s = sentences[i]
    print("iteration:",i)
    kl_summary = summarize(each_s,False)
    lda_summary = summarize(each_s,True)
    
    es.index(index='index_20ng_summaries', doc_type='20ng_summaries', id=i, body={
        'doc_id': i,
        'doc_text': each_s,
        'doc_kl_sum': kl_summary,
        'doc_lda_sum': lda_summary
    })
    
    

iteration: 0
iteration: 1
iteration: 2
iteration: 3


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

iteration: 4
iteration: 5


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14


...

iteration: 892
iteration: 893
iteration: 894
iteration: 895
iteration: 896
iteration: 897


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

iteration: 898
iteration: 899
iteration: 900
iteration: 901
iteration: 902
iteration: 903
iteration: 904


/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

KeyboardInterrupt: 

In [144]:
r = es.search(index='index_20ng_summaries',doc_type='20ng_summaries')

In [147]:
print(r)

{'took': 52, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': 87, 'max_score': 1.0, 'hits': [{'_index': 'index_20ng_summaries', '_type': '20ng_summaries', '_id': '0', '_score': 1.0, '_source': {'doc_id': 0, 'doc_text': ['I am a little confused on all of the models of the 88-89 bonnevilles.', 'I have heard of the LE SE LSE SSE SSEI.', 'Could someone tell me the differences are far as features or performance.', 'I am also curious to know what the book value is for prefereably the 89 model.', 'And how much less than book value can you usually get them for.', 'In other words how much are they in demand this time of year.', 'I have heard that the mid-spring early summer is the best time to buy.'], 'doc_kl_sum': ['And how much less than book value can you usually get them for.', 'In other words how much are they in demand this time of year.', 'Could someone tell me the differences are far as features or performance.', 'I have heard of

In [ ]:
###DUC dataset

In [154]:
def get_gold_summary(filename):
    summary_filename = filename.lower()+".txt" 
    if not os.path.exists(path +'/Summaries/'+summary_filename):
        return ''
    else:
        fd = open(path +'/Summaries/'+summary_filename, "r+")
        fileCon = fd.read()
        #print(fileCon)
        return fileCon

In [155]:
import os
def get_filenames(folder):

    filenames= os.listdir(folder)
    return filenames

path = "Downloads/DUC2001/"
filenames = get_filenames(path)

def get_text(content):
    #print(content)
    doc_text = []
    done = False
    for i in range(len(content)):
        #line = line.strip() # skip the empty line
        line = content[i].lstrip()
        #line = line.replace('\n','')
        
        if line == '<TEXT>\n':
            
            for j in range(i+1,len(content)):
                if content[j] == '</TEXT>\n':
                    done == True
                    break
                doc_text.append(content[j].strip())
        if done:
            break
    #print(doc_text)
    return doc_text

fileCon = ''
i = 0
documents_duc = []
gold_summary = []
exclude = ['annotations.txt','.DS_Store','kartest','Summaries','notes.txt']
for filename in filenames:
    if filename in exclude :
        continue
    #filename = 'FBIS-41815'
    fd = open(path +'/'+ filename, "r+")
           
    fileCon = fd.readlines()
    fd.close()
    #print(fileCon)
    text = get_text(fileCon)
    text = ''.join(text)
    documents_duc.append(text)
    gold_sum = get_gold_summary(filename)
    gold_summary.append(gold_sum)
    
    

In [156]:
sentences_duc = get_sentences(documents_duc)
KL_summaries_duc = []
LDA_summaries_duc = []
for i in range(len(sentences_duc)):
    each_s = sentences_duc[i]
    kl_summary = summarize(each_s,False)
    lda_summary = summarize(each_s,True)
    es.index(index='index_duc_summaries', doc_type='duc_summaries', id=i, body={
        'doc_id': i,
        'doc_text': each_s,
        'doc_kl_sum': kl_summary,
        'doc_lda_sum': lda_summary,
        'doc_gold_sum':gold_summary[i]
    })
    KL_summaries_duc.append(kl_summary)
    LDA_summaries_duc.append(lda_summary)
    
    

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

...

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online

In [218]:
def write_to_file(content,path):
    f = open(path,"w")
    f.write(content)
    f.close()

In [233]:
os.mkdir("summaries")

In [234]:
for i in range(len(KL_summaries_duc)):
    content = KL_summaries_duc[i]
    model_path = "summaries/model_klsummaries_"+str(i)
    os.mkdir(model_path)
    model_filename = model_path+'/'+"text_"+str(i)+"_"+str(i)+".txt"
    if not content:
        write_to_file('',model_filename)
    else:
        write_to_file(''.join(content),model_filename)
    

In [235]:
for i in range(len(gold_summary)):
    content = gold_summary[i]
    system_path = "summaries/gold_klsummaries_"+str(i)
    os.mkdir(system_path)
    sys_filename = system_path+'/'+"text_"+str(i)+'.txt'
    if not content:
        write_to_file('',sys_filename)
    else:
        write_to_file(content,sys_filename)

In [236]:
from pyrouge import Rouge155

In [237]:
def rouge_summaries(system_,model_):    
    r = Rouge155()
    r.system_dir = system_
    r.model_dir = model_
    # r.system_filename_pattern = 'text.(\d+).txt'
    # r.model_filename_pattern = 'text.[A-Z].#ID#.txt'
    out_result = []

    system_filename_pattern = "text_(\d+).txt"
    model_filename_pattern = "text_(\d+)_(\d+).txt"
    r.system_filename_pattern = system_filename_pattern
    r.model_filename_pattern = model_filename_pattern
    print("--------file "+str(i)+"-----------------")
    output = r.convert_and_evaluate()
    print(output)

    output_dict = r.output_to_dict(output)
    print(output_dict)

In [239]:
for i in range(len(KL_summaries_duc)):
    rouge_summaries("summaries/gold_klsummaries_"+str(i),"summaries/model_klsummaries_"+str(i))

2018-04-10 02:26:35,021 [MainThread  ] [INFO ]  Writing summaries.
2018-04-10 02:26:35,023 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpu5uap63t/system and model files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpu5uap63t/model.
2018-04-10 02:26:35,024 [MainThread  ] [INFO ]  Processing files in summaries/gold_klsummaries_0.
2018-04-10 02:26:35,025 [MainThread  ] [INFO ]  Processing text_0.txt.
2018-04-10 02:26:35,027 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpu5uap63t/system.
2018-04-10 02:26:35,028 [MainThread  ] [INFO ]  Processing files in summaries/model_klsummaries_0.
2018-04-10 02:26:35,029 [MainThread  ] [INFO ]  Processing text_0_0.txt.
2018-04-10 02:26:35,031 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpu5uap63t/model.
2018-04-10 02:26:35,034 [MainThread  ] [INFO ]  Written ROUGE

--------file 0-----------------


2018-04-10 02:26:35,227 [MainThread  ] [INFO ]  Writing summaries.
2018-04-10 02:26:35,229 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpyogpuljl/system and model files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpyogpuljl/model.
2018-04-10 02:26:35,230 [MainThread  ] [INFO ]  Processing files in summaries/gold_klsummaries_1.
2018-04-10 02:26:35,232 [MainThread  ] [INFO ]  Processing text_1.txt.
2018-04-10 02:26:35,234 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpyogpuljl/system.
2018-04-10 02:26:35,236 [MainThread  ] [INFO ]  Processing files in summaries/model_klsummaries_1.
2018-04-10 02:26:35,237 [MainThread  ] [INFO ]  Processing text_1_1.txt.
2018-04-10 02:26:35,240 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpyogpuljl/model.
2018-04-10 02:26:35,242 [MainThread  ] [INFO ]  Written ROUGE

---------------------------------------------
1 ROUGE-1 Average_R: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-1 Average_P: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-1 Average_F: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
---------------------------------------------
1 ROUGE-2 Average_R: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-2 Average_P: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-2 Average_F: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
---------------------------------------------
1 ROUGE-3 Average_R: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-3 Average_P: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-3 Average_F: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
---------------------------------------------
1 ROUGE-4 Average_R: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-4 Average_P: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
1 ROUGE-4 Average_F: 0.00000 (95%-conf.int. 0.00000 - 0.00000)
---------------------------------------------
1 ROUGE-L Aver

2018-04-10 02:26:36,019 [MainThread  ] [INFO ]  Writing summaries.
2018-04-10 02:26:36,020 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp1f_0vvcz/system and model files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp1f_0vvcz/model.
2018-04-10 02:26:36,022 [MainThread  ] [INFO ]  Processing files in summaries/gold_klsummaries_2.
2018-04-10 02:26:36,025 [MainThread  ] [INFO ]  Processing text_2.txt.
2018-04-10 02:26:36,027 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp1f_0vvcz/system.
2018-04-10 02:26:36,028 [MainThread  ] [INFO ]  Processing files in summaries/model_klsummaries_2.
2018-04-10 02:26:36,030 [MainThread  ] [INFO ]  Processing text_2_2.txt.
2018-04-10 02:26:36,032 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp1f_0vvcz/model.
2018-04-10 02:26:36,034 [MainThread  ] [INFO ]  Written ROUGE

...

2018-04-10 02:30:11,818 [MainThread  ] [INFO ]  Writing summaries.
2018-04-10 02:30:11,819 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp3qrmh50m/system and model files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp3qrmh50m/model.
2018-04-10 02:30:11,820 [MainThread  ] [INFO ]  Processing files in summaries/gold_klsummaries_184.
2018-04-10 02:30:11,821 [MainThread  ] [INFO ]  Processing text_184.txt.
2018-04-10 02:30:11,825 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp3qrmh50m/system.
2018-04-10 02:30:11,826 [MainThread  ] [INFO ]  Processing files in summaries/model_klsummaries_184.
2018-04-10 02:30:11,829 [MainThread  ] [INFO ]  Processing text_184_184.txt.
2018-04-10 02:30:11,832 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmp3qrmh50m/model.
2018-04-10 02:30:11,835 [MainThread  ] [INFO ]  Wri

---------------------------------------------
1 ROUGE-1 Average_R: 0.91111 (95%-conf.int. 0.91111 - 0.91111)
1 ROUGE-1 Average_P: 0.04940 (95%-conf.int. 0.04940 - 0.04940)
1 ROUGE-1 Average_F: 0.09372 (95%-conf.int. 0.09372 - 0.09372)
---------------------------------------------
1 ROUGE-2 Average_R: 0.77273 (95%-conf.int. 0.77273 - 0.77273)
1 ROUGE-2 Average_P: 0.04101 (95%-conf.int. 0.04101 - 0.04101)
1 ROUGE-2 Average_F: 0.07789 (95%-conf.int. 0.07789 - 0.07789)
---------------------------------------------
1 ROUGE-3 Average_R: 0.65116 (95%-conf.int. 0.65116 - 0.65116)
1 ROUGE-3 Average_P: 0.03382 (95%-conf.int. 0.03382 - 0.03382)
1 ROUGE-3 Average_F: 0.06430 (95%-conf.int. 0.06430 - 0.06430)
---------------------------------------------
1 ROUGE-4 Average_R: 0.54762 (95%-conf.int. 0.54762 - 0.54762)
1 ROUGE-4 Average_P: 0.02781 (95%-conf.int. 0.02781 - 0.02781)
1 ROUGE-4 Average_F: 0.05293 (95%-conf.int. 0.05293 - 0.05293)
---------------------------------------------
1 ROUGE-L Aver

2018-04-10 02:30:12,981 [MainThread  ] [INFO ]  Writing summaries.
2018-04-10 02:30:12,982 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpgz13e61t/system and model files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpgz13e61t/model.
2018-04-10 02:30:12,983 [MainThread  ] [INFO ]  Processing files in summaries/gold_klsummaries_185.
2018-04-10 02:30:12,984 [MainThread  ] [INFO ]  Processing text_185.txt.
2018-04-10 02:30:12,987 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpgz13e61t/system.
2018-04-10 02:30:12,988 [MainThread  ] [INFO ]  Processing files in summaries/model_klsummaries_185.
2018-04-10 02:30:12,990 [MainThread  ] [INFO ]  Processing text_185_185.txt.
2018-04-10 02:30:12,992 [MainThread  ] [INFO ]  Saved processed files to /var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpgz13e61t/model.
2018-04-10 02:30:12,994 [MainThread  ] [INFO ]  Wri

---------------------------------------------
1 ROUGE-1 Average_R: 1.00000 (95%-conf.int. 1.00000 - 1.00000)
1 ROUGE-1 Average_P: 0.01188 (95%-conf.int. 0.01188 - 0.01188)
1 ROUGE-1 Average_F: 0.02348 (95%-conf.int. 0.02348 - 0.02348)
---------------------------------------------
1 ROUGE-2 Average_R: 0.81818 (95%-conf.int. 0.81818 - 0.81818)
1 ROUGE-2 Average_P: 0.00892 (95%-conf.int. 0.00892 - 0.00892)
1 ROUGE-2 Average_F: 0.01765 (95%-conf.int. 0.01765 - 0.01765)
---------------------------------------------
1 ROUGE-3 Average_R: 0.60000 (95%-conf.int. 0.60000 - 0.60000)
1 ROUGE-3 Average_P: 0.00595 (95%-conf.int. 0.00595 - 0.00595)
1 ROUGE-3 Average_F: 0.01178 (95%-conf.int. 0.01178 - 0.01178)
---------------------------------------------
1 ROUGE-4 Average_R: 0.44444 (95%-conf.int. 0.44444 - 0.44444)
1 ROUGE-4 Average_P: 0.00397 (95%-conf.int. 0.00397 - 0.00397)
1 ROUGE-4 Average_F: 0.00787 (95%-conf.int. 0.00787 - 0.00787)
---------------------------------------------
1 ROUGE-L Aver

CalledProcessError: Command '['/Users/mallika2493/pyrouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl', '-e', '/Users/mallika2493/pyrouge/tools/ROUGE-1.5.5/data', '-c', '95', '-2', '-1', '-U', '-r', '1000', '-n', '4', '-w', '1.2', '-a', '-m', '/var/folders/d0/vtcq_yxd4_l06mjb3n9wmb0m0000gn/T/tmpj_pyfzir/rouge_conf.xml']' returned non-zero exit status 255.

In [215]:
type(output)

str